In [9]:
import cv2
import numpy as np

def kmeans(k, data):
    #  cluster centroids are randomly initailized
    centroids = data[np.random.choice(data.shape[0], k, replace=True)] 
#replace=True means same index can be retrived many times to enhance randomnes & diff size handling of cluster
    while True:
        # Allocating every data point to nearest centroid
        distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)#axis=2 element wise cal b/w datapoints & centeriod
        labels = np.argmin(distances, axis=1)#get min distance,axis=1 bcz findinfg centriod with min distance

        # Update centroids by taking the mean of the assigned data points 
        new_centroids = []
        for i in range(k):#iteration for checking any data points assigned to the current cluster index i
            if np.sum(labels == i) > 0:
                new_centroid = data[labels == i].mean(axis=0) #cal mean if true retuned from above statement
            else:
                new_centroid = data[np.random.choice(data.shape[0])]#if false returned from above>randomly choose datapoint
            new_centroids.append(new_centroid)
        new_centroids = np.array(new_centroids)

        # Check if centroids have converged
        if np.all(centroids == new_centroids):
            break
 # Check if centroids have converged/or close to each other then break
        centroids = new_centroids

    return labels, centroids




def cal_Likelihood(pixels, clusters, centroids):
    num_clusters = clusters.shape[0]#get size of 1st dimension of self.centriod arr
    likelihood = np.zeros((pixels.shape[0], num_clusters))#no of pixels and no of clusters

    for k in range(num_clusters):
        distance = np.linalg.norm(pixels - centroids[k], axis=1)#cal eculidean distance b/w pixels & centriod
        likelihood[:, k] = np.exp(-distance)#Smaller distance result in better likelihood 

    return likelihood



def Applying_Segmentation(test_image_path, seed_image_path, N):
    # for Loading test image and seed image
    test_image = cv2.imread(test_image_path)
    seed_image = cv2.imread(seed_image_path)

    # for display purpose
    display_image = test_image.copy()

    # Extract pixels of seed from seed image
    seed_pixels = []
    mouse_event = False#indicates no seed is selected

    def Mouse_events(event, x, y, flags, param):
        nonlocal mouse_event
        if event == cv2.EVENT_LBUTTONDOWN:
            mouse_event = True
            seed_pixels.append((x, y))
            cv2.circle(display_image, (x, y), 4, (255, 255, 255), -1)
            cv2.imshow("Select seed pixels", display_image)

    cv2.namedWindow("Select seed pixels")
    cv2.setMouseCallback("Select seed pixels", Mouse_events)

    while True:
        cv2.imshow("Select seed pixels", display_image)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            break

    cv2.destroyAllWindows()

    if not mouse_event or len(seed_pixels) == 0:
        raise ValueError("No seed pixels found!")

    # Converting the seed pixels to numpy array
    seed_pixels = np.array(seed_pixels)

    # Perform k-means clustering on the seed pixels
    k = 64  # Number of clusters
    labels, centroids = kmeans(k, seed_pixels)

    # Select N clusters for foreground and background
    selected_clusters = np.unique(labels)[:N]

    # Compute likelihoods for each pixel
    Height, Width = test_image.shape[:2]
    pixels = test_image.reshape(-1, 3)
    likelihoods = cal_Likelihood(pixels, selected_clusters, centroids)

    # Assign each pixel to the class with higher likelihood
    value = np.argmax(likelihoods, axis=1).reshape(Height, Width).astype(np.uint8) * 255

    return value



In [10]:
test_img_path = 'C:\\Users\\Dell\\Desktop\\Part 2 Attachments\\Select seed pixels.png'
seed_img_path = 'C:\\Users\\Dell\\Desktop\\Part 2 Attachments\\van Gogh stroke.png'
N = 64  # Number of clusters for each class


In [11]:
segmentation = Applying_Segmentation(test_img_path, seed_img_path, N)

ValueError: No seed pixels found!